Combining the google and apple datasets

Purpose
This notebook is going to bring together the google and apple datasets. 

In [1]:
from os.path import exists, isfile
import random
import time

import re
import math

import pandas as pd
import numpy as np
import math

In [2]:
save_path = '../../datasets/2300_combine_kaggle_datasets.csv'

In [3]:
if not exists("../../datasets/2200_clean_apple.csv"):
    print ("Missing dataset file")
    
df_apple=pd.read_csv("../../datasets/2200_clean_apple.csv")
df_apple.head()

,apple_id,apple_title,apple_size,apple_price,apple_reviews,apple_rating,apple_pegi,apple_genre,normed_apple_rating,z_score_apple,log_apple_reviews
0,281656475,PAC-MAN Premium,96.119141,3.99,21292,4.0,Everyone,Games,0.8,-0.083987,4.328216
1,281796108,Evernote - stay organized,151.232422,0.00,161065,4.0,Everyone,Business,0.8,-0.083987,5.207001
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",95.867188,0.00,188583,3.5,Everyone,Others,0.7,-0.806018,5.275503
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",122.558594,0.00,262241,4.0,Teen,Lifestyle,0.8,-0.083987,5.418701
4,282935706,Bible,88.476562,0.00,985920,4.5,Everyone,Books & Reference,0.9,0.638043,5.993842


In [4]:
if not exists("../../datasets/2100_clean_google.csv"):
    print ("Missing dataset file")
    
df_google=pd.read_csv("../../datasets/2100_clean_google.csv")
df_google.head()

,google_title,google_genre,google_rating,google_reviews,google_size,google_price,google_pegi,log_google_reviews,normed_google_rating,z_score_google
0,Photo Editor & Candy Camera & Grid & ScrapBook,Utilities,4.1,159,19.0,0.0,Everyone,2.201397,0.82,-0.143135
1,Coloring book moana,Utilities,3.9,967,14.0,0.0,Everyone,2.985426,0.78,-0.537987
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Utilities,4.7,87510,8.7,0.0,Everyone,4.942058,0.94,1.041422
3,Sketch - Draw & Paint,Utilities,4.5,215644,25.0,0.0,Teen,5.333737,0.90,0.646570
4,Pixel Draw - Number Art Coloring Book,Utilities,4.3,967,2.8,0.0,Everyone,2.985426,0.86,0.251718


Title to title mappings

The same apps can have different titles when they are released in different platforms. This happens when the title of each app is in the format like "A - B" or "A : B". However, some apps have titles like "A - B" or "A : B" but they are different. For example, different games in a series can have titles like "A - B", "A - C", and "A - D" but they cannot be considered as the same app.

A possible solution to this problem is, first take "A" out of "A - B" or "A : B" and save it as the key to combine two datasets. If such a key is unique in the new dataset, we simply consider it as a perfect match. However, when such a key is not unique, we only consider those matches with the same google title and apple title as the correct match.

In [5]:
def clean_title(x):
    x = str(x)
    pos1 = x.find('-')
    pos2 = x.find('–')
    pos3 = x.find(':')
    pos4 = x.find('(')
    if pos1 != -1:
        x = x[:pos1].strip()
    if pos2 != -1:
        x = x[:pos2].strip()
    if pos3 != -1:
        x = x[:pos3].strip()
    if pos4 != -1:
        x = x[:pos4].strip()
    r1 = '[’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    x = re.sub(r1, '', x)
    x = x.strip()
    return x

In [6]:
df_google['trim_title'] = df_google['google_title'].apply(clean_title)
df_apple['trim_title'] = df_apple['apple_title'].apply(clean_title)

In [7]:
combine_apps = df_apple.set_index('trim_title').join(
    df_google.set_index('trim_title'), how='inner').reset_index()
combine_apps.shape

(781, 22)

In [8]:
same_title = combine_apps[(combine_apps.duplicated('trim_title') == True) & (combine_apps['google_title'] == combine_apps['apple_title'])]
same_trim = combine_apps[combine_apps.duplicated('trim_title') == False]
combine_apps = same_title.append(same_trim)
combine_apps.sample(10)

,trim_title,apple_id,apple_title,apple_size,apple_price,apple_reviews,apple_rating,apple_pegi,apple_genre,normed_apple_rating,...,google_title,google_genre,google_rating,google_reviews,google_size,google_price,google_pegi,log_google_reviews,normed_google_rating,z_score_google
423,NBA,335744614,NBA,106.882812,0.00,43682,3.5,Everyone,Health & Fitness,0.7,...,NBA,Health & Fitness,4.4,108318,35.0,0.0,Everyone,5.034701,0.88,0.449144
506,Pokémon GO,1094591345,Pokémon GO,277.292969,0.00,257627,3.0,Everyone 10+,Games,0.6,...,Pokémon GO,Games,4.1,10424925,85.0,0.0,Everyone,7.018073,0.82,-0.143135
303,Grim Tales,1057673686,Grim Tales: The Final Suspect - A Hidden Objec...,767.910156,6.99,86,4.5,Everyone 10+,Games,0.9,...,Grim Tales: The Wishes CE,Games,4.3,4923,20.0,0.0,Everyone 10+,3.692230,0.86,0.251718
108,Camera360,443354861,"Camera360 - Selfie Filter Camera, Photo Editor",153.693359,0.00,16729,4.5,Teen,Social Networking,0.9,...,Camera360: Selfie Photo Editor with Funny Sticker,Books & Reference,4.3,4865093,51.0,0.0,Everyone,6.687091,0.86,0.251718
385,Meitu,416048305,Meitu,135.518555,0.00,6478,5.0,Everyone,Social Networking,1.0,...,"Meitu – Beauty Cam, Easy Photo Editor",Books & Reference,4.5,462702,45.0,0.0,Everyone,5.665301,0.90,0.646570
436,Nextdoor,640360962,Nextdoor,78.708008,0.00,14402,4.5,Everyone,Social Networking,0.9,...,Nextdoor - Local neighborhood news & classifieds,Social Networking,4.3,51502,20.0,0.0,Teen,4.711824,0.86,0.251718
56,BET NOW,841118013,BET NOW - Watch Shows,36.232422,0.00,967,2.5,Mature 17+,Entertainment,0.5,...,BET NOW - Watch Shows,Entertainment,4.2,14807,19.0,0.0,Teen,4.170467,0.84,0.054292
312,Hangman,286911400,Hangman.,4.544922,0.00,42316,3.0,Everyone 10+,Games,0.6,...,Hangman,Others,4.4,23302,12.0,0.0,Everyone,4.367393,0.88,0.449144
678,Toy Blast,890378044,Toy Blast,256.082031,0.00,75822,4.5,Everyone,Games,0.9,...,Toy Blast,Games,4.7,1889250,NaN,0.0,Everyone,6.276289,0.94,1.041422
432,Narcos,1143052259,Narcos: Cartel Wars,170.600586,0.00,4490,4.5,Teen,Games,0.9,...,Narcos: Cartel Wars,Others,4.7,244797,52.0,0.0,Teen,5.388806,0.94,1.041422


In [9]:
combine_apps.shape

(604, 22)

The difference of each app's two ratings is calculated here.

In [10]:
combine_apps['z_score_google_sub_apple'] = combine_apps['z_score_google'] - combine_apps['z_score_apple']
combine_apps.sample(5)

,trim_title,apple_id,apple_title,apple_size,apple_price,apple_reviews,apple_rating,apple_pegi,apple_genre,normed_apple_rating,...,google_genre,google_rating,google_reviews,google_size,google_price,google_pegi,log_google_reviews,normed_google_rating,z_score_google,z_score_google_sub_apple
47,Animal Jam,1003820457,Animal Jam - Play Wild!,591.586914,0.0,26990,4.5,Everyone 10+,Games,0.9,...,Others,4.6,361970,58.0,0.0,Everyone,5.558673,0.92,0.843996,0.205953
468,PAC,1102508135,PAC-MAN Pop,235.743164,0.0,1000,4.5,Everyone,Games,0.9,...,Others,4.3,24877,54.0,0.0,Everyone,4.395798,0.86,0.251718,-0.386326
315,Hay Day,506627515,Hay Day,108.376953,0.0,567344,4.5,Everyone,Games,0.9,...,Others,4.5,10053186,94.0,0.0,Everyone,7.002304,0.90,0.646570,0.008527
713,WEBDE Mail,368948250,WEB.DE Mail,97.202148,0.0,168,4.0,Everyone,Business,0.8,...,Social Networking,4.3,226541,NaN,0.0,Everyone,5.355147,0.86,0.251718,0.335705
536,SeatGeek,582790430,"SeatGeek – Tickets to Sports, Concerts & Broadway",65.037109,0.0,6088,4.5,Everyone,Entertainment,0.9,...,News,4.4,15558,26.0,0.0,Everyone,4.191954,0.88,0.449144,-0.188899


In [11]:
combine_apps['norm_google_sub_apple'] = combine_apps['normed_google_rating'] - combine_apps['normed_apple_rating']
combine_apps.sample(5)

,trim_title,apple_id,apple_title,apple_size,apple_price,apple_reviews,apple_rating,apple_pegi,apple_genre,normed_apple_rating,...,google_rating,google_reviews,google_size,google_price,google_pegi,log_google_reviews,normed_google_rating,z_score_google,z_score_google_sub_apple,norm_google_sub_apple
616,Tango,372513032,"Tango - Free Video Call, Voice and Chat",128.601562,0.0,75412,4.5,Mature 17+,Social Networking,0.9,...,4.3,3806669,NaN,0.0,Mature 17+,6.580545,0.86,0.251718,-0.386326,-0.04
286,Google Chrome,535886823,Google Chrome – The Fast and Secure Web Browser,90.966797,0.0,55750,3.5,Mature 17+,Utilities,0.7,...,4.3,9642995,NaN,0.0,Everyone,6.984212,0.86,0.251718,1.057736,0.16
688,Truecaller,448142450,Truecaller - Spam Identification & Block,80.496094,0.0,27791,4.5,Everyone,Utilities,0.9,...,4.5,7820209,NaN,0.0,Everyone,6.893218,0.90,0.646570,0.008527,0.00
342,Instagram,389801252,Instagram,108.675781,0.0,2161558,4.5,Teen,Social Networking,0.9,...,4.5,66577313,NaN,0.0,Teen,7.823326,0.90,0.646570,0.008527,0.00
228,FOX NOW,571096102,FOX NOW - Watch Full Episodes and Stream Live TV,54.334961,0.0,39391,4.5,Everyone,Entertainment,0.9,...,3.9,60841,NaN,0.0,Teen,4.784196,0.78,-0.537987,-1.176030,-0.12


We only keep the common features for the new dataset.

In [12]:
use_cols = [
    'apple_id', 'trim_title', 'apple_title', 'apple_genre', 'apple_rating',
       'apple_reviews', 'apple_size', 'apple_pegi',
       'normed_apple_rating', 'google_title', 'google_rating',
       'google_reviews', 'google_size', 'google_price', 
       'normed_google_rating', 'log_google_reviews', 'log_apple_reviews',
    'z_score_google', 'z_score_apple', 'z_score_google_sub_apple', 'norm_google_sub_apple'
]

df = combine_apps[use_cols].copy()
df.head()

,apple_id,trim_title,apple_title,apple_genre,apple_rating,apple_reviews,apple_size,apple_pegi,normed_apple_rating,google_title,...,google_reviews,google_size,google_price,normed_google_rating,log_google_reviews,log_apple_reviews,z_score_google,z_score_apple,z_score_google_sub_apple,norm_google_sub_apple
107,898968647,Call of Duty®,Call of Duty®: Heroes,Games,4.5,179416,201.075195,Teen,0.9,Call of Duty®: Heroes,...,1604146,57.0,0.00,0.88,6.205244,5.253861,0.449144,0.638043,-0.188899,-0.02
170,1147297267,Dont Starve,Don't Starve: Shipwrecked,Games,3.5,495,604.341797,Everyone 10+,0.7,Don't Starve: Shipwrecked,...,1468,4.9,4.99,0.82,3.166726,2.694605,-0.143135,-0.806018,0.662884,0.12
223,352670055,F,F-Sim Space Shuttle,Games,4.5,6403,72.855469,Everyone,0.9,F-Sim Space Shuttle,...,5427,NaN,4.99,0.88,3.734560,3.806384,0.449144,0.638043,-0.188899,-0.02
301,763692274,Grand Theft Auto,Grand Theft Auto: San Andreas,Games,4.0,32533,1964.965820,Mature 17+,0.8,Grand Theft Auto: San Andreas,...,348962,26.0,6.99,0.88,5.542778,4.512324,0.449144,-0.083987,0.533131,0.08
355,771989093,LEGO® Friends,LEGO® Friends,Games,4.0,400,730.941406,Everyone,0.8,LEGO® Friends,...,854,6.9,4.99,0.88,2.931458,2.602060,0.449144,-0.083987,0.533131,0.08


In [13]:
df.columns = ['apple_id','trim_title', 'apple_title', 'genre', 'apple_rating',
       'apple_reviews', 'apple_size', 'pegi',
       'normed_apple_rating', 'google_title', 'google_rating',
       'google_reviews', 'google_size', 'price', 
       'normed_google_rating', 'log_google_reviews', 'log_apple_reviews',
    'z_score_google_rating', 'z_score_apple_rating', 'z_score_google_sub_apple','norm_google_sub_apple']
df.shape

(604, 21)

In [14]:
df['google_revenue'] = df['google_reviews'] * df['price']
df['apple_revenue'] = df['apple_reviews'] * df['price']

df.loc[df['google_revenue']>0,'log_google_revenue'] = df[df['google_revenue']>0]['google_revenue'].apply(lambda x: math.log(x, 10))
df.loc[df['apple_revenue']>0,'log_apple_revenue'] = df[df['apple_revenue']>0]['apple_revenue'].apply(lambda x: math.log(x, 10))

In [15]:
df.to_csv(save_path, index=False)
df.shape

(604, 25)